<H1>Numerisk løsning av den tidsuavhengig Schrödingerligningen</H1>

<p>Notatboken og tilhørende kildekode ligger på <a href="https://github.com/kaprests/Numerical-1D-TISE">GitHub.</a></p>

<H2>Gruppe 21: Henrik E. V. Skaldebø og Knut Andre G. Prestsveen</H2>

<H2>Sammendrag</H2>
<p>Denne notatboken prosjekt nummer to i faget TMA4320, og øving i faget TFY4215 ved Norges tekniske og naturvitenskapelige universitet. Den tar for seg nummerisk løsning av den tidsuavhengige Schrödingerligningen (TUSL) for ikke-vekselvirkende elektroner i ulike potensialer som modellerer partikkel i boks, atomer, molekyler og krystaller i én dimensjon.</p>

<H2>Introduksjon</H2>
<p>What we do, why it is cool and how it wørks</p>
<p>Vi ser som nevnt på løsninger av TUSL én dimensjon, for ikke-vekselvirkende elektroner, som også adlyder Pauliprinsippet (write some moar later)</p>

<h3>Numerisk løsning av (1D) TUSL</h3>
TUSL i en dimensjon er

**TUSL**,

Det taes utgangspunkt i bokspotensialet, altså at partikkelen kan oppholde seg på et endelig område langs x-aksen, avgrenset av harde vegger(uendelig potensiale) i endepunktene $x=0$ og $x=L$. Partikkelens posisjon på dette området diskretiseres i $N$ mulige verdier, slik at løsningene går fra $\psi(x)$ til $\psi(n\Delta x)$, hvor $n = 0, 1, 2, ... , N, N+1$. I endepunktene er bølgefunskjonen null på grunn av kontinuitet i overgangen ved veggene, siden bølgefunksjonen må være null der potensialet er uendelig. Med diskret posisjon og operatorer kan TUSL da skrives som de $N$ differanseligningene

**TUSL diskret**.

Som igjen kan skrives på matriseform

**TUSL matrise**; n = 1,2,3, ... , N

hvor $\mathbf{H}$ er $NxN$ matrise med diagonalelementer $MAIN_DIAG_YO$ og elementene $secon_diag_yo$ over og under diagonalen, og $E$ er energi-/egenverdier. Denne egenverdiligningen har ikke-trivielle løsninger kun når $det(put in determ eqn)$, hvor $I$ er identitetsmatrisen. $E\mathbf{I}$ er da en $NxN$ matrise med energiverdiene langs diagonalen, og poenget er at de numeriske løsningene for bølgefunksjonene og energinivåene er gitt av hhv. egenvektorene og egenverdiene til $\mathbf{H}$-matrisen.





In [ ]:
#kode

import numpy as np
from scipy.linalg import eigh_tridiagonal
from matplotlib import pyplot as plt

w = 0.1
b = 0.05
n = 1000
m = 1
h_bar = 1
num_well = 4

# Box potential
V_box = lambda x: 0*x

# n well potential
def well(x, w, n_w, b, L,V0=-3000):
	n = len(x)
	V_mid_ex = np.array([[V0]*int(w*n/L) + [0]*int(b*n/L)]*n_w).flatten()
	V_mid = V_mid_ex[:-int(b*n/L)]
	
	m = len(V_mid)
	diff = n - m
	len_b = diff//2
	len_f = diff - len_b
	V_front = np.array([0]*len_f)
	V_back = np.array([0]*len_b)

	V_temp = np.append(V_front, V_mid)
	V_vec = np.append(V_temp, V_back)

	return V_vec


def analyze(n_w):
	L = (20 + n_w)*w + (n_w -1)*b
	delta_x = L/(n+1)
	x_vec = np.linspace(0, L, n)


	#H = np.zeros([n, n])
	main_diag = np.zeros(n)
	off_diag = np.zeros(n-1)
		
	if n_w == 0:
		V_vals = V_box(x_vec)
		iter_lim = 5

	else:
		V_vals = well(x_vec, w, n_w, b, L)
		iter_lim = n_w*3

	for i in range(n):
		# Fills main diagonal
		#H[i][i] = ((h_bar**2)/(m*(delta_x**2))) + V(x_vec[i])
		main_diag[i] = ((h_bar**2)/(m*(delta_x**2))) + V_vals[i]

	for i in range(n-1):
		# Fills second diagonals below and above main
		#H[i+1][i] = H[i][i +1] = (h_bar**2)/(2*m*(delta_x**2))
		off_diag[i] = -(h_bar**2)/(2*m*(delta_x**2))

	energies, wave_funcs = eigh_tridiagonal(main_diag, off_diag)
	wave_funcs = wave_funcs.T
	return energies, wave_funcs, iter_lim, L, V_vals


def band_widths(lower, upper):
	x_vec = np.linspace(lower, upper, upper-lower)
	bw = np.zeros([3, upper-lower])
	for i in range(lower, upper):
		energies, _,_,_,_ = analyze(i)
		#print("heyhey: ", energies, "\n")
		for j in range(3):
			bw[j][i-lower] = np.absolute(energies[i*j] - energies[(j+1)*i -1])

	for i in range(3):
		plt.plot(x_vec, bw[i])
	plt.show()
    

def plot_wave_funcs(energies, wave_funcs, iter_lim, L, V_vals):
	x_vec = np.linspace(0, L, len(V_vals))
	for i in range(iter_lim): 
		plt.plot([0, L], [energies[i]]*2)
		plt.plot(x_vec, V_vals)
		plt.plot(x_vec, energies[i] + wave_funcs[i]*3000)
	plt.show()


E_b, w_b, iter_lim_b, L_b, V_vals_b = analyze(0)
plot_wave_funcs(E_b, w_b, iter_lim_b, L_b, V_vals_b)
energies, wave_funcs, iter_lim, L, V_vals = analyze(5)
plot_wave_funcs(energies, wave_funcs, iter_lim, L, V_vals)

band_widths(2, 50)